In [141]:
import random
import string
import datetime
import numpy as np
import pandas as pd

### Analisis y confeccion de agregados de datos de usuarios

In [142]:
usuarios = pd.read_csv("MOCK_DATA.csv")

In [143]:
subareas = {
    "Logística":["Gestión de inventario","Transporte y distribución","Optimización de rutas","Gestión de almacenes","Logística inversa"],
    "Producción":["Control de calidad","Cadena de suministro","Mejora de procesos","Planificación","Mantenimiento"],
    "Planificación":["Planificación estratégica","Capacidad","Planificación de proyectos"],
    "Calidad":["Control de calidad","Gestión de riesgos","Mejora continua","Auditoría interna"],
    "Financias":["Contabilidad y registros financieros","Gestión de tesorería"],
    "Mantenimiento":["Mantenimiento correctivo","Gestión de activos"] 
}

In [144]:

registroCWID = []
def generate_unique_word(text):
    word = ''.join(random.choices(string.ascii_uppercase, k=6))
    while word in registroCWID:
        word = ''.join(random.choices(string.ascii_uppercase, k=6))
    registroCWID.append(word)
    return word

usuarios["CWID"] = usuarios["Nombre"].apply(generate_unique_word)


In [145]:
def plantEmployee(empresa):
    if empresa == "Unge" or empresa == "Hays":
        return "Local"
    else:
        return "Externo"

In [146]:
usuarios["Externo"] = usuarios["Empresa"].apply(plantEmployee)

In [147]:
empresas = usuarios.drop_duplicates(subset="Empresa")[["Empresa","Externo"]]
empresas["EID"] = empresas["Empresa"].apply(generate_unique_word)

In [148]:
def asigSubArea (sector):
    return random.choice(subareas[sector])

In [149]:
usuarios["SubArea"] = usuarios["Área"].apply(asigSubArea)

In [150]:
subareastabla = usuarios.drop_duplicates(subset="SubArea")[["Área","SubArea"]]
subareastabla["SAID"] = subareastabla["SubArea"].apply(generate_unique_word)
subareastabla = subareastabla[["Área","SubArea","SAID"]]

In [151]:
usuarios = usuarios.merge(empresas, how='left', on='Empresa')
usuarios = usuarios.merge(subareastabla, how='left', on='SubArea')
usuarios = usuarios[["Nombre","Apellido","Genero","CWID","EID","SAID"]]

,Nombre,Apellido,Genero,CWID,EID,SAID
0,Maurise,Kaindl,Male,GHZJTZ,GMVCVJ,JJBJWJ
1,Dall,Havick,Male,WJTRJS,MSRPRL,IOWZDU
2,Melisande,Bywaters,Female,XBWZWG,BLSDMV,LECZFR
3,Martelle,Bendall,Female,TJVMBD,GMVCVJ,LVNIHQ
4,Jacenta,Virr,Female,EKJNGS,BLSDMV,VKFKEN
...,...,...,...,...,...,...
205,Gasparo,Tole,Male,KELKAC,GMVCVJ,WECLGY
206,Vance,McGeever,Male,HVSCPV,MSRPRL,FZYGSZ
207,Minda,Balazot,Female,AOPCNY,SATHBL,NXZCDZ
208,Kerrie,Tapin,Female,OQIZSN,MSRPRL,XXXCOP


### Creacion de funciones aleatoreas para la simulacion de registros

In [153]:

# Función para generar fechas aleatorias de lunes a viernes entre 2020 y 2022
def generate_random_weekday_date():
    year = random.randint(2020, 2022)
    month = random.randint(1, 12)
    day = random.randint(1, 28)
    hour = random.randint(8, 16)
    minute = random.randint(0, 59)
    second = random.randint(0, 59)
    date = datetime.datetime(year, month, day, hour, minute, second)
    while date.weekday() > 4:
        year = random.randint(2020, 2022)
        month = random.randint(1, 12)
        day = random.randint(1, 28)
        hour = random.randint(8, 16)
        minute = random.randint(0, 59)
        second = random.randint(0, 59)
        date = datetime.datetime(year, month, day, hour, minute, second)
    return date

In [154]:
# Función para seleccionar al azar un elemento de una lista y asegurarse de que no sea anterior al elemento anteriormente seleccionado
def select_random_item(lst):
    selected_item = random.choice(lst)
    while True:
        new_item = random.choice(lst)
        if new_item >= selected_item:
            return [new_item,selected_item]

### Creacion de tabla registro

In [155]:
Practicas = ["Mejor enfocada", "Gestión visual", "trabajo en equipo", "5S", "Mindset digital"]
Motivos = ["flexibilidad", "liderazgo", "integridad", "eficiencia"]

In [156]:
registros = []
for n in range(3000):
    users = select_random_item(list(usuarios["CWID"])) 
    registros.append({
        "id": n+1,
        "nominador": users[0],
        "nominado": users[1],
        "práctica": random.choice(Practicas),
        "motivo": random.choice(Motivos),
        "fecha": generate_random_weekday_date()
    })

In [157]:
data = pd.DataFrame(registros)

In [158]:
data.to_csv("Registros.csv",index=False)
usuarios.to_csv("Usuarios.csv",index=False)
empresas.to_csv("Empresas.csv",index=False)
subareastabla.to_csv("SubAreas.csv",index=False)

### Analisis de contenido entre tabla y metadatos

In [159]:
from metadataCreator import Metadata

In [160]:
metadata = Metadata()
metadata.ReadCSV("Usuarios.csv")
metadata.ReadCSV("Registros.csv")
metadata.ReadCSV("Empresas.csv")
metadata.ReadCSV("SubAreas.csv")
metadata.CreateMetadata(title="Reconocimientos")

In [161]:
metadata.SaveMetadata()